# Example showing how to access, edit and navigate assertional knowledge.
## Testing cardinality/constrtaints



In [1]:
from simphony_osp.namespaces import city, emmo, rdfs
from simphony_osp.tools import pretty_print


In [2]:
# instantiate ontology individuals

person = city.Citizen(
    name="Martin",
    age=15,
)

Multi-class individuals. This is a complication, as now person has two `name` attributes, one from each name space.
Meanning we cannot use the '.name' notation any longer. Hence, it is a better practice to use 

`person[city['name']])`

In [3]:
person.classes = city.Citizen, emmo.Cogniser  

In [4]:
 freiburg = city.City(name="Freiburg", coordinates=[47.997791, 7.842609])

In [5]:
neighborhoods = {
    city.Neighborhood(name=name, coordinates=coordinates)
    for name, coordinates in [
        ("Altstadt", [47.99525, 7.84726]),
        ("Stühlinger", [47.99888, 7.83774]),
        ("Neuburg", [48.00021, 7.86084]),
        ("Herdern", [48.00779, 7.86268]),
        ("Brühl", [48.01684, 7.843]),
    ]
}

In [6]:
citizen_1 = city.Citizen(name="Nikola", age=35)
citizen_2 = city.Citizen(name="Lena", age=70)

In [7]:
pretty_print(freiburg)

- Ontology individual:
  identifier: 9fe7b4d7-7685-42a5-899b-06bb87335d78
  type: City (city )
  superclasses: City (city ), Populated Place (city ), Geographical Place (city ), Thing (owl )
  values:  coordinates: [47.997791  7.842609]
           name: Freiburg


In [8]:
freiburg.superclasses

[<OntologyClass: City https://www.simphony-osp.eu/city#City>,
 <OntologyClass: Populated Place https://www.simphony-osp.eu/city#PopulatedPlace>,
 <OntologyClass: Geographical Place https://www.simphony-osp.eu/city#GeographicalPlace>,
 <OntologyClass: Thing http://www.w3.org/2002/07/owl#Thing>]

In [9]:
# edit relationships, attributes and/or annotations
freiburg[city.hasPart] |= neighborhoods

In [10]:
freiburg[city.hasInhabitant] += citizen_1, citizen_2, person

In [11]:
freiburg[rdfs.comment] = "A city in the southwest of Germany."

In [12]:
freiburg.get(rel=city.hasInhabitant, oclass=city.Citizen)

{<OntologyIndividual: ee7a9e66-089c-462e-a70e-df5150e6a89e>, <OntologyIndividual: df73ff4a-ef3c-48c6-a222-0a5a49d95a49>, <OntologyIndividual: 9676fbea-0c1f-4186-ac0d-7f830cb6a33b>} <has inhabitant of ontology individual 9fe7b4d7-7685-42a5-899b-06bb87335d78>

In [13]:
for citizen in freiburg.get(rel=city.hasInhabitant, oclass=city.Citizen):
    pretty_print(citizen)

- Ontology individual:
  identifier: df73ff4a-ef3c-48c6-a222-0a5a49d95a49
  types: Cogniser (emmo ), Citizen (city )
  superclasses: Cogniser (emmo ), Interpreter (emmo ), CausalSystem (emmo ), CausalStructure (emmo ), Item (emmo ), EMMO (emmo ), Thing (owl ), SemioticEntity (emmo ), Semiotics (emmo ), Perspective (emmo ), Citizen (city ), Person (city ), Living Being (city )
  values:  age: 15
           name: Martin
- Ontology individual:
  identifier: ee7a9e66-089c-462e-a70e-df5150e6a89e
  type: Citizen (city )
  superclasses: Citizen (city ), Person (city ), Living Being (city ), Thing (owl )
  values:  age: 70
           name: Lena
- Ontology individual:
  identifier: 9676fbea-0c1f-4186-ac0d-7f830cb6a33b
  type: Citizen (city )
  superclasses: Citizen (city ), Person (city ), Living Being (city ), Thing (owl )
  values:  age: 35
           name: Nikola


In [14]:
citizen_1.age, citizen_1.name


(35, 'Nikola')

In [15]:

citizen_2.attributes


mappingproxy({<OntologyAttribute: name https://www.simphony-osp.eu/city#name>: frozenset({'Lena'}),
              <OntologyAttribute: age https://www.simphony-osp.eu/city#age>: frozenset({70})})

`person.name`

this does not work as the dot notation does not suppoert multiple values

```
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[29], line 1
----> 1 person.name
      2 # this does not work as the dot notation does not suppoert multiple values

File ~/dev/simphony_4_git/simphony-osp/simphony_osp/ontology/individual.py:931, in OntologyIndividual.__getattr__(self, name)
    922 elif num_attributes >= 2:
    923     error = (
    924         f"There are multiple attributes with label or suffix {name} "
    925         f"associated with {self}:"
   (...)
    929         f"values of this attribute."
    930     )
--> 931     raise AttributeError(error)
    932 attr = attributes.pop()
    934 values = self.attributes_value_generator(attr)

AttributeError: There are multiple attributes with label or suffix name associated with df73ff4a-ef3c-48c6-a222-0a5a49d95a49: https://www.simphony-osp.eu/city#name, http://xmlns.com/foaf/0.1/name. Please use an OntologyAttribute object together with the indexing notation `individual[attribute]` to access the values of this attribute.
```


In [20]:
X=person._attributes_get_by_name('name')
print(type(X))
X_1 = X.pop()
print(X_1)
X_1
X_1.__str__()


<class 'set'>
name


'name'

In [30]:
# print(person["name"].all())
# this does not work as intended based on the current  doc


# The best way to access multiple attributes
Use the dictionary/array like form, as it allows one to directly specify the proper ontology. Though for the future a better way is needed. 

In [24]:
for citizen in freiburg.get(rel=city.hasInhabitant, oclass=city.Citizen):
    print(citizen[city['name']])

{'Martin'}
{'Lena'}
{'Nikola'}


In [25]:
 city['name']

<OntologyAttribute: name https://www.simphony-osp.eu/city#name>

In [26]:
# navigate the assertional knowledge
can_drive = {
   frozenset(citizen[city['name']]): "Yes" if (True if citizen.age >= 18 else False) else "No"
    for citizen in freiburg.get(rel=city.hasInhabitant, oclass=city.Citizen)
}

In [27]:
for name, drives in can_drive.items():
    print(f"Can {name} drive? {drives}")
    

Can frozenset({'Martin'}) drive? No
Can frozenset({'Lena'}) drive? Yes
Can frozenset({'Nikola'}) drive? Yes
